In [1]:
import pandas as pd
from statistics import mean 
import math 
import numpy as np
from itertools import islice

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [2]:
# Question ID and Answer ID pair
qid_docid = pd.read_csv("train/FiQA_train_question_doc_final.tsv", sep="\t")
qid_docid = qid_docid [['qid', 'docid']]

In [4]:
# Create dict for question id and relevant passgages
# keys: query ids, values: list of relevant passages
qid_rel = {}

for index, row in qid_docid.iterrows():
    
    if row['qid'] not in qid_rel:
        qid_rel[row['qid']] = []
    qid_rel[row['qid']].append(row['docid'])
    
take(10, qid_rel.items())

[(0, [18850]),
 (1, [14255]),
 (2, [308938]),
 (3, [296717, 100764, 314352, 146317]),
 (4, [196463]),
 (5, [69306]),
 (6, [560251, 188530, 564488]),
 (7, [411063]),
 (8, [566392, 65404]),
 (9, [509122, 184698])]

In [5]:
# Number of relevant passages for each query
num_rel = [len(v) for v in qid_rel.values()]

avg_num_rel = mean(num_rel)
max_num_rel = max(num_rel)
min_num_rel = min(num_rel)

print("Average number of relevant passages for each query: {}\n".format(avg_num_rel))
print("Max number of relevant passages for each query: {}\n".format(max_num_rel))
print("Min number of relevant passages for each query: {}\n".format(min_num_rel))

Average number of relevant passages for each query: 2.5737063778580023

Max number of relevant passages for each query: 23

Min number of relevant passages for each query: 1



In [16]:
# Answer Ranking for each question
doc_ranking = pd.read_csv("fiqa-passage/run2_train.tsv", sep="\t", header=None)
doc_ranking = doc_ranking.rename(columns={1: 'qid', 2: 'doc_id', 3:'rank'})
doc_ranking.head(5)

,0,qid,doc_id
0,0,531578,1
1,0,417981,2
2,0,324911,3
3,0,524879,4
4,0,397608,5


In [17]:
print("Number of candidate answers for all questions: {}".format(len(doc_ranking)))

Number of candidate answers for all questions: 6638656


In [18]:
# Create dict for query id and ranked candidates
# key: query ids, values: list of 1000 ranked candidates
qid_ranked_docs = {}

with open("fiqa-passage/run2_train.tsv",'r') as f:
    for line in f:
        # [qid, doc_id, rank]
        line = line.strip().split('\t')
        qid = int(line[0])
        doc_id = int(line[1])
        rank = int(line[2])
        
        if qid not in qid_ranked_docs:
            # Create a list of size 1000 for each query to store the candidates
            candidates = [0]*1000
            qid_ranked_docs[qid] = candidates
        qid_ranked_docs[qid][rank-1] = doc_id
        
#take(1, qid_ranked_docs.items())

In [19]:
# Helper functions for evaluation
def get_rel_score(rel_score, cand_docs, rel_docs, k):
    """
    Returns a dictionary of the top-k relevancy scores of docs in the candidate answers
    
    key - question id
    value - list of relevancy scores with 1 being relevant and 0 being irrelevant
    
    Example: {0: [0, 1, 0], 1: [1, 1, 0]}
    """
    if qid not in rel_score:
        rel_score[qid] = []

        for i in range(0, k):
            if cand_docs[i] in rel_docs:
                rel_score[qid].append(1)
            else:
                rel_score[qid].append(0)

    return rel_score

def dcg(rels, k):
    """
    Discounted Cumulative Gain
    
    Returns the cumulated DCG of the top-k relevant docs across all queries
    """
    cumulated_sum = rels[0]
    for i in range(1, k):
        cumulated_sum += rels[i]/math.log(i+1,2)
    return cumulated_sum

def avg_ndcg(rel_score):
    """
    Average Normalized Discounted Cumulative Gain
    
    Computes the DCG, iDCG, and nDCG for each query
    
    Returns the averyage nDCG across all queries
    """
    ndcg_list = []
    for qid, rels in rel_score.items():
        dcg_val = dcg(rels, k)   
        sorted_rel = sorted(rels, reverse=True)
        idcg_val = dcg(sorted_rel, k)

        try:
            ndcg_val = dcg_val/idcg_val
            ndcg_list.append(ndcg_val)
        except ZeroDivisionError:
            ndcg_list.append(0)
            
    assert len(ndcg_list) == len(rel_score), "Relevant score doesn't match"

    avg = mean(ndcg_list)

    return avg

def compute_RR(cand_docs, rel_docs, cumulated_reciprocal_rank, rank_pos, k):
    """
    Computes the reciprocal rank - probability of correctness of rank
    
    Returns the cumulated reciprocal rank across all queries and the
    positions of the relevant docs in the candidates
    """
    
    for i in range(0, k):
        # If the doc_id of the top k ranked candidate passages is in the list of relevant passages
        if cand_docs[i] in rel_docs:
            # Compute the reciprocal rank (i is the ranking)
            rank_pos.append(i+1)
            cumulated_reciprocal_rank += 1/(i+1)
            break
            
    return cumulated_reciprocal_rank, rank_pos

In [20]:
# print(qid_ranked_docs[6][:10])
# print(qid_rel[6][:10])
# print()
# print(qid_ranked_docs[102][:10])
# print(qid_rel[102][:10])

### Evaluation of BM25

In [26]:
# Evaluate top-1000 candidates
k = 1000
cumulated_reciprocal_rank = 0
num_rel_docs = 0
rel_score = {}
precision_list = {}
rank_pos = []

# For each query
for qid in qid_ranked_docs:
    # If the query has a relevant passage
    if qid in qid_rel:
        # Get the list of relevant docs for a query
        rel_docs = qid_rel[qid]
        # Get the list of ranked docs for a query
        cand_docs = qid_ranked_docs[qid]
        # Compute relevant scores of the candidates
        rel_scores = get_rel_score(rel_score, cand_docs, rel_docs, k)
        # MRR@k
        cumulated_reciprocal_rank, r_pos = compute_RR(cand_docs, rel_docs, cumulated_reciprocal_rank, rank_pos, k)
        

print("Average nDCG@{} for {} queries: {}".format(k, len(qid_rel), avg_ndcg(rel_scores)))
print()

MRR = cumulated_reciprocal_rank/len(qid_rel)

print("MRR@{} for {} queries: {}\n".format(k, len(qid_rel), MRR))

# Mean precision at k
k = 1
scores = {}
precision_at_k = []
for qid, scores in rel_scores.items():
    num_rel = 0
    for i in range(0, k):
        if scores[i] == 1:
            num_rel += 1
    precision_at_k.append(num_rel/k)     
mean_precision_at_k = mean(precision_at_k)

print("Average Precision@{}: {}".format(k, mean_precision_at_k))

Average nDCG@1000 for 6648 queries: 0.37228219563855214

MRR@1000 for 6648 queries: 0.3191620897939914

Average Precision@1: 0.2378158844765343


In [22]:
print("The lowest rank of the first relevant passage: {}\n".format(max(r_pos)))
print("The highest rank of the first relevant passage: {}\n ".format(min(r_pos)))
print("The average rank of the first relevant passage {}\n".format(mean(r_pos)))

The lowest rank of the first relevant passage: 1000

The highest rank of the first relevant passage: 1
 
The average rank of the first relevant passage 81.76945491943778



In [24]:
# Check if any candidate passages don't contain any relevant passages
counter = 0
for qid, scores in rel_scores.items():
    np_scores = np.asarray(scores)
    
    if np.all(np_scores==0):
        counter += 0

print("Number of candidate passage that don't contrain any relevant passages: {}".format(counter))

Number of candidate passage that don't contrain any relevant passages: 0
